### Define helper functionality to visualize forward and inverse image mapping

In [1]:
import os

from pathlib import Path
from typing import Callable

import Metashape
import numpy as np

from mynd.backend import metashape as backend
from mynd.collections import CameraGroup, StereoCameraGroup
from mynd.geometry import HitnetModel, load_hitnet
from mynd.image import Image, filter_image_clahe

from mynd.tasks.export_stereo import ExportStereoTask, invoke_stereo_export_task

from mynd.utils.environment import Environment, load_environment
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


GroupID = CameraGroup.Identifier


def create_clahe_filter(clip: float, size: int) -> Callable:
    """Creates a CLAHE filter with the given parameters."""

    def clahe_filter(image: Image) -> Image:
        return filter_image_clahe(image, clip=clip, size=size)

    return clahe_filter


def filter_disparity_map(disparity: np.ndarray) -> np.ndarray:
    """Filter a disparity map."""

    # TODO: Add disparity filter

    return disparity


def prepare_task_config(
    identifier: GroupID,
    stereo_camera: StereoCameraGroup,
    range_directory: Path,
    normal_directory: Path,
) -> ExportStereoTask.Config:
    """Prepares a stereo export task config by creating setting up paths, creating directories,
    and loading a disparity model."""

    environment: Environment = load_environment().unwrap()

    model: HitnetModel = load_hitnet(
        environment.resource_directory
        / Path("hitnet_models/hitnet_eth3d_720x1280.onnx")
    ).unwrap()

    # NOTE: Debug only!
    logger.info(f"Camera pairs: {len(stereo_camera.camera_pairs)}")
    logger.info(f"Image loaders: {len(stereo_camera.image_loaders)}")

    clahe_filter = create_clahe_filter(clip=6.0, size=10)

    config = ExportStereoTask.Config(
        range_directory=range_directory / Path(f"{identifier.label}_ranges"),
        normal_directory=normal_directory / Path(f"{identifier.label}_normals"),
        calibrations=stereo_camera.calibrations,
        camera_pairs=stereo_camera.camera_pairs,
        image_loaders=stereo_camera.image_loaders,
        # Processors
        disparity_estimator=model,
        image_filter=clahe_filter,
        disparity_filter=filter_disparity_map,
    )

    # Create export directories if they do not exist
    if not config.range_directory.exists():
        os.mkdir(config.range_directory)
    if not config.normal_directory.exists():
        os.mkdir(config.normal_directory)

    return config


def batch_stereo_export(
    target: GroupID,
    stereo_groups: list[StereoCameraGroup],
    range_directory: Path,
    normal_directory: Path,
) -> None:
    """Iterate over the stereo groups in a camera group."""
    for stereo_group in stereo_groups:

        config: ExportStereoTask.Config = prepare_task_config(
            target, stereo_group, range_directory, normal_directory
        )

        result: Result[ExportStereoTask.Result, str] = (
            invoke_stereo_export_task(config)
        )


def main():
    """Main function."""

    DOCUMENT_PATH: Path = Path(
        "/data/kingston_snv_01/acfr_metashape_projects_dev/r23m7ms0_lite_with_metadata.psz"
    )

    RANGE_DIRECTORY: Path = Path("/data/kingston_snv_01/acfr_stereo_ranges")
    NORMAL_DIRECTORY: Path = Path("/data/kingston_snv_01/acfr_stereo_normals")

    DEPLOYMENT: str = "r23m7ms0_20120601_070118"

    # r23685bc_lite_metadata.psz
    # r23685bc_20100605_021022, r23685bc_20120530_233021, r23685bc_20140616_225022

    # r23m7ms0_lite_with_metadata.psz
    # r23m7ms0_20100606_001908, r23m7ms0_20120601_070118, r23m7ms0_20140616_044549

    match backend.load_project(DOCUMENT_PATH):
        case Ok(None):
            pass
        case Err(message):
            logger.error(message)

    groups: dict[str, GroupID] = {
        group.label: group for group in backend.get_group_identifiers().unwrap()
    }

    target: GroupID = groups.get(DEPLOYMENT)

    match backend.camera_services.retrieve_stereo_cameras(target):
        case Ok(stereo_camera_groups):
            batch_stereo_export(
                target,
                stereo_camera_groups,
                range_directory=RANGE_DIRECTORY,
                normal_directory=NORMAL_DIRECTORY,
            )
        case Err(message):
            logger.error(message)


# ---------- Invoke main function ----------
main()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


LoadProject: path = /data/kingston_snv_01/acfr_metashape_projects_dev/r23m7ms0_lite_with_metadata.psz
loaded project in 22.8026 sec


2024-10-20 20:14:57.352679745 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 20 Memcpy nodes are added to the graph tf2onnx for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2024-10-20 20:14:57.424 | INFO     | __main__:prepare_task_config:58 - Camera pairs: 3733
2024-10-20 20:14:57.424 | INFO     | __main__:prepare_task_config:59 - Image loaders: 7466
2024-10-20 20:14:57.425 | INFO     | mynd.geometry.stereo_rectification:_compute_rectified_calibrations:179 - [[1.80545996e+03 0.00000000e+00 6.78856567e+02]
 [0.00000000e+00 1.77432092e+03 5.23316162e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


NotImplementedError: 